In [ ]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline

In [ ]:
import mr

In [ ]:
from mrcnn import model as modellib
from mrcnn import visualize

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return fig, ax

In [ ]:
work_path = os.path.join("E:", os.sep, "RCNNTanks256Valid")
os.chdir(work_path)

In [ ]:
dataset = mr.MRDataset()
dataset.load(work_path)
dataset.prepare()

In [ ]:
inference_config = mr.InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference",
                          config=inference_config,
                          model_dir="logs")

find_last = False
if find_last:
    model_path = model.find_last()
    print("Loading weights from ", model_path)
else:
    model_path = "mask_rcnn_mr_0316.h5"
model.load_weights(model_path, by_name=True)

In [ ]:
image_id = random.choice(dataset.image_ids)

original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, inference_config, 
                           image_id, use_mini_mask=False)

results = model.detect([original_image], verbose=0)
r = results[0]
fig, (ax1, ax2) = get_ax(1,2)
visualize.display_instances(original_image,
                            gt_bbox,
                            gt_mask,
                            gt_class_id, 
                            dataset.class_names,
                            ax=ax1)

visualize.display_instances(original_image,
                            r['rois'],
                            r['masks'],
                            r['class_ids'], 
                            dataset.class_names,
                            r['scores'],
                            ax=ax2)

In [ ]:
fig.savefig("fig-005.png")